In [1]:
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import json
import os
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs


In [5]:
db_df = pd.read_csv('/home/valesco/Datasets/data_broker_urls1.csv')
db_df.head()

,names,url,search_url,terms_of_service,privacy_policy,opt_out
0,plusaddress,http://plusaddress.com/,http://plusaddress.com/results?cx=partner-pub-...,http://plusaddress.com/#terms,http://plusaddress.com/#privacy,null
1,1-800-US-Search,http://www.ussearch.com/consumer/index.jsp,https://www.ussearch.com/search/people/[firstn...,http://www.ussearch.com/consumer/commerce/abou...,http://www.ussearch.com/consumer/commerce/abou...,"Opt-Out: Yes, http://www.ussearch.com/consumer..."
2,10 Digits,http://10digits.us,https://whitepages.plus/n/[firstname]_[middlei...,http://10digits.us/terms,http://10digits.us/privacy,null
3,411.com (a WhitePages.com company),http://www.411.com,https://www.411.com/name/[firstname]-[lastname...,http://www.whitepagescustomers.com/terms-of-se...,http://www.whitepagescustomers.com/data-policy/,null
4,4Addresses.com,http://www.4addresses.com,http://4addresses.net/Search.php?q1=[firstname...,http://4addresses.com/TermsOfUse.php,http://4addresses.com/PrivacyPolicy.php,null


In [10]:
search_urls = db_df['search_url'].values

cust_info = {
    'firstname': 'Kathleen',
    'middlename': 'Elizabeth',
    'middleinitial': 'E',
    'lastname': 'Petersen',
    'streetaddress': '16175+Feliciana+Ave',
    'streetname': 'Felciana+Ave',
    'streetnumber': '16175',
    'city': 'Prarieville',
    'statename': 'Louisiana', 
    'stateabbreviation': 'LA',
    'zipcode': '70769',
    'sex': 'Female',
    'phonenumber': ''
    }

document_dict = []
master_ls = []

for url in search_urls:
    if 'http' in url:
        ls = re.findall('\[([A-Za-z0-9_]+)\]', url)
        master_ls.extend(ls)
        
        new_url = url
        for key in ls:
            new_url = new_url.replace('[{}]'.format(key), cust_info[key])
            
        print(new_url)
        browser = webdriver.Chrome()
        browser.get(new_url)

        elements = browser.find_elements_by_xpath('//*')
        
        document = ''
        
        for element in elements:
            try:
                if len(element.text) > 0:
                    document += element.text
            except:
                pass

        time.sleep(1)
        browser.close()
        
        document_name = db_df['names'].loc[db_df['search_url'] == url].values[0]
        
        temp_dict = {}
        temp_dict['name'] = document_name
        temp_dict['doc'] = document
        temp_dict['url'] = new_url
        
        document_dict.append(temp_dict)
        


http://plusaddress.com/results?cx=partner-pub-5929267120303720%3A1975859289&cof=FORID%3A10&ie=UTF-8&q=Kathleen+Petersen+Louisiana&sa=Search&siteurl=plusaddress.com%2F&ref=&ss=9039j4292519j44
https://www.ussearch.com/search/people/Kathleen/~/Petersen/LA
https://whitepages.plus/n/Kathleen_E_Petersen/Prarieville_LA/e454cbeb96819870bcc6aaad5f6fa110
https://www.411.com/name/Kathleen-Petersen/Prarieville-LA
http://4addresses.net/Search.php?q1=Kathleen&q2=Petersen&t=Louisiana&first=0
https://www.addresssearch.com/results.php?type=address&address=16175+Feliciana+Ave&location=Prarieville%2C+LA
http://www.addresses.com/people/Kathleen+Petersen/Prarieville+LA
https://www.advancedbackgroundchecks.com/search/results.aspx?type=&fn=Kathleen&mi=E&ln=Petersen&age=&city=Prarieville&state=Louisiana
http://www.ypstate.com/info/LA/Prarieville/zip/70769 *needstobescraped
https://www.archives.com/search/vital/results?FirstName=Kathleen&LastName=Petersen&Country=United%20States&Region=Louisiana&cert=False#r
h

In [13]:
result_dict = []
for temp_dict in document_dict:
    string = temp_dict['doc']
    count = 0
    for key, value in cust_info.items():
        value = value.replace('+', ' ')
        if value in string:
            count += 1
    
    result_temp_dict = {}
    result_temp_dict['name'] = temp_dict['name']
    result_temp_dict['url'] = temp_dict['url']
    result_temp_dict['count'] = count
    
    result_dict.append(result_temp_dict)
    
    
for result in result_dict:
    if result['count'] > 5:
        print(result['count'], result['url'])

7 https://www.ussearch.com/search/people/Kathleen/~/Petersen/LA
7 https://whitepages.plus/n/Kathleen_E_Petersen/Prarieville_LA/e454cbeb96819870bcc6aaad5f6fa110
7 http://www.addresses.com/people/Kathleen+Petersen/Prarieville+LA
7 https://www.advancedbackgroundchecks.com/search/results.aspx?type=&fn=Kathleen&mi=E&ln=Petersen&age=&city=Prarieville&state=Louisiana
6 https://www.peoplesmart.com/background-results?FirstName=Kathleen&LastName=Petersen&State=LA&AFF=158&tid=UWP&&utm_source=UWP&utm_medium=affiliate&utm_campaign=UWP&utm_content=UWPFunnel?FirstName=Kathleen&LastName=Petersen&State=LA&AFF=158&tid=UWP&&utm_source=UWP&utm_medium=affiliate&utm_campaign=UWP&utm_content=UWPFunnel
7 https://www.instantpeoplefinder.com/find-people/Kathleen-Petersen/LA
6 https://www.truthfinder.com/results/?firstName=Kathleen&lastName=Petersen&middleInitial=&city=&state=LA&age=&gender=&theme=&type=&qLocation=false&qRelatives=false&qOver30=false
7 https://www.intelius.com/people-search/Kathleen-Petersen/LA
